## LionAGI - 103

**Reflective Iterative editing** of a piece of text

and 

**Structured Output**

In [1]:
from IPython.display import display, Markdown


def printmd(string):
    display(Markdown(string))

In [2]:
from lionagi import iModel, Branch

config = {
    "provider": "openai",
    "api_key_schema": "OPENAI_API_KEY",
}

gpt4o = iModel(model="gpt-4o", **config)

Introducing custom logic to LLM flow

---

Now we can do something more complex other than having LLMs following through a pre-defined flow. We can introduce custom logic to the LLM flow. 

But first let's take a look at `structured_output`


LLM works by taking a bunch of texts and generate a bunch of texts. Natural Language Processing (NLP) isn't necessarily easy to to help us make use of the response, and one way to mitigate that is **structured output**

going back to the comedian example

In [3]:
comedian_system_message = "a respected zoo keeper, also a comedian who knows very well about indian post modern literature"
critic_system_message = "As an a young professor, you are trying to get into the prestigious circle of literature criticism."

In [4]:
branch = Branch(system=comedian_system_message, imodel=gpt4o)

In [5]:
prompt1 = "Tell a joke: a blue whale met a big white shark in a bar, then started dancing."

prompt2 = "Tell a joke: an elephant knocked off a banana tree."

response = await branch.chat(prompt1)
printmd(response)

Sure, here's a light-hearted take:

A blue whale and a big white shark walk into a bar. The bartender looks up and says, "Wow, we don't get many of your kind in here!" 

The blue whale replies, "Well, we heard the drinks here are killer!" 

The shark nods and adds, "And I heard the dance floor is a real splash!"

So, they both hit the dance floor, and the whale starts doing the worm. The shark looks over and says, "Hey, I thought you were more of a 'wave' kind of dancer!"

The whale chuckles and says, "Nah, I just like to keep things current!"

And with that, they both danced the night away, making waves and having a whale of a time!

now intead of asking comedian to give a plain joke, let's instruct it to also give a few other things, by giving an example to the comedian.

In [6]:
example = {
    "category": "jokes",
    "topic": "assigning personalities to animals",
    "interpretation": "The..... therefore.....",
    "joke": "...",
    "rating": "an integer from 1 to 10",
}

response = await branch.chat(instruction=prompt2, requested_fields=example)

In [7]:
from lionfuncs import as_readable_json

print(as_readable_json(response))

{
    "category": "jokes",
    "topic": "assigning personalities to animals",
    "interpretation": "The joke plays on the idea of elephants being large and clumsy, therefore humorously causing unintended chaos.",
    "joke": "Why did the elephant knock off the banana tree? Because it wanted to make a little 'a-peel' in the jungle!",
    "rating": 7
}


In [8]:
response["rating"]

7

now we can see that the comedian can give us a joke, a joke type, and a joke rating. By making the output a `dict` we can effectively use the output of the comedian. 

We asked it to give itself a rating, here is an example to use it: 

ask comedian to repeat fixing this joke until it reaches **8/10**

In [9]:
async def reflective_joke(prompt, max_attempts=3):

    response = await branch.chat(
        instruction=prompt,
        requested_fields=example,
    )

    print(as_readable_json(response))

    while response["rating"] < 8 and max_attempts > 0:
        print("-------------- Not a great joke, trying again --------------")
        response = await branch.chat(
            instruction="try again",
            requested_fields=example,
        )
        print(as_readable_json(response))
        max_attempts -= 1

    if response["rating"] >= 8:
        print("-------------- Great joke! --------------")
        return response

In [10]:
await reflective_joke(prompt2)

{
    "category": "jokes",
    "topic": "assigning personalities to animals",
    "interpretation": "The joke highlights the elephant's size and strength, therefore humorously attributing its clumsiness to its massive presence.",
    "joke": "Why did the elephant knock off the banana tree? It just couldn't resist a little 'trunk' show!",
    "rating": 6
}
-------------- Not a great joke, trying again --------------
{
    "category": "jokes",
    "topic": "assigning personalities to animals",
    "interpretation": "The joke plays on the elephant's reputation for being both strong and somewhat clumsy, therefore humorously causing unintended consequences.",
    "joke": "Why did the elephant knock off the banana tree? It wanted to start a 'fruitful' conversation!",
    "rating": 7
}
-------------- Not a great joke, trying again --------------
{
    "category": "jokes",
    "topic": "assigning personalities to animals",
    "interpretation": "The joke uses the elephant's size and strength t

{'category': 'jokes',
 'topic': 'assigning personalities to animals',
 'interpretation': "The joke uses the elephant's size and strength to create a humorous scenario, therefore attributing its actions to its playful nature.",
 'joke': "Why did the elephant knock off the banana tree? It was just trying to make a 'smoothie' entrance!",
 'rating': 8}